In [545]:
import arff
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

In [635]:
### NOTE: The only methods you are required to have are:
#   * predict
#   * fit
#   * score
def entropy(p):
    if p > 0:
        return -p * np.log2(p)
    else:
        return 0

class DTClassifier(BaseEstimator,ClassifierMixin):

    def __init__(self):
        """ Initialize class with chosen hyperparameters.
        Args:
            hidden_layer_widths (list(int)): A list of integers which defines the width of each hidden layer
            lr (float): A learning rate / step size.
            shuffle: Whether to shuffle the training data each epoch. DO NOT SHUFFLE for evaluation / debug datasets.
        Example:
            DT  = DTClassifier()
        """
        self.graph = {():[]}

    def fit(self, X, y):
        """ Fit the data; Make the Desicion tree
        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
            y (array-like): A 2D numpy array with the training targets
        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)
        """
        X = X.astype(int)
        y = y.flatten().astype(int)
        self.num_features = X.shape[1]
        self.feature_class = X.max(axis=0) + 1
        self.num_class = int(y.max()) + 1
        self._split(X, y, [], (), ())
        self.root = (self.graph[()][0],)
        return self
    
    def _split(self, X, y, mask, parent, path):
        gain = self.calc_gain(X, y, mask)
        split_on = gain.argmin()
        node = path + (split_on,)
        self.graph[parent].append(split_on)
        self.graph[node] = []
        mask.append(split_on)
        F_split = X[:, split_on]
        for i in range(self.feature_class[split_on]):
            y_p = y[F_split==i]
            if len(y_p) == 0 or len(mask) > self.num_features:
                vals, counts = np.unique(y, return_counts=True)
                ind = counts.argmax()
                self.graph[node].append([vals[ind]])
            elif len(set(y_p)) == 1:
                y_val = set(y_p).pop()
                self.graph[node].append([y_val])
            else:
                new_path = path + ((split_on, i),)
                self._split(X[F_split==i], y[F_split==i], mask.copy(), node, new_path)
        
    def calc_gain(self, X, y, mask):
        n = X.shape[0]
        gain = []
        for i in range(self.num_features):
            if i in mask:
                gain.append(np.inf)
                continue
            F = X[:, i]
            P = np.vstack((F, y)).T
            info = 0
            for j in range(int(F.max()) + 1):
                d = sum(F==j)
                ratio = d / n
                if d == 0:
                    continue
                curr_entropy = sum(entropy(sum(P[F==j][:,-1]==k) / d) for k in range(self.num_class))
                info += ratio*curr_entropy
            gain.append(info)
        return np.array(gain)
        
    def predict(self, X):
        """ Predict all classes for a dataset X
        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
        Returns:
            array, shape (n_samples,)
                Predicted target values per element in X.
        """
        X = X.astype(int)
        y_hat = []
        for x in X:
            y_hat.append(self._find(x, self.root))
        return np.array(y_hat).reshape(-1, 1)

    def _find(self, x, node):
        z = x[node[-1]]
        v = self.graph[node][x[node[-1]]]
        if type(v) == list:
            return v[0]
        else:
            node = node[:-1] + ((node[-1], z),) + (v,)
            return self._find(x, node)
    
    def score(self, X, y):
        """ Return accuracy of model on a given dataset. Must implement own score function.
        Args:
            X (array-like): A 2D numpy array with data, excluding targets
            y (array-li    def _shuffle_data(self, X, y):
        """
        y = y.astype(int)
        y_hat = self.predict(X)
        return (y_hat == y).sum() / len(y)


# Debug

In [636]:
lenses = arff.Arff(arff=r'lenses.arff', label_count=3)
all_lenses = arff.Arff(arff=r'all_lenses.arff', label_count=3)
X_train = lenses[:, :-1].astype(int)
y_train = lenses[:, -1].reshape(-1, 1).astype(int)
X_test = all_lenses[:, :-1].astype(int)
y_test = all_lenses[:, -1].reshape(-1, 1).astype(int)
dt = DTClassifier()
dt.fit(X_train, y_train)
pred = dt.predict(X_test)
acc = dt.score(X_test, y_test)
np.savetxt("pred_lenses2.csv", pred, delimiter=',')
print("Accuracy = [{:.2f}]".format(acc))

Accuracy = [0.33]


In [638]:
lenses = arff.Arff(arff=r'lenses.arff', label_count=3)
all_lenses = arff.Arff(arff=r'all_lenses.arff', label_count=3)
X = lenses[:, :-1].astype(int)
y = lenses[:, -1].reshape(-1, 1).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
dt = DTClassifier()
dt.fit(X_train, y_train)
pred = dt.predict(X_test)
acc = dt.score(X_test, y_test)
acc

0.75

# Evaluation

In [639]:
zoo = arff.Arff(arff=r'zoo.arff', label_count=7)
all_zoo = arff.Arff(arff=r'all_zoo.arff', label_count=7)
X_train = zoo[:, :-1].astype(int)
y_train = zoo[:, -1].reshape(-1, 1).astype(int)
X_test = all_zoo[:, :-1].astype(int)
y_test = all_zoo[:, -1].reshape(-1, 1).astype(int)
dt = DTClassifier()
dt.fit(X_train, y_train)
pred = dt.predict(X_test)
acc = dt.score(X_test, y_test)
np.savetxt("pred_zoo.csv", pred, delimiter=',')
print("Accuracy = [{:.2f}]".format(acc))

Accuracy = [0.15]


# Cars

In [640]:
cars = arff.Arff(arff=r'cars.arff', label_count=4)
np.random.shuffle(cars[:])
X = cars[:, :-1]
y = cars[:, -1].reshape(-1, 1)
dt = DTClassifier()
cv_results = cross_validate(dt, X, y, scoring='accuracy', return_train_score=True, cv=10)
cv_results

{'fit_time': array([0.38768101, 0.38428402, 0.40556908, 0.37191892, 0.40394902,
        0.40693474, 0.37606621, 0.39530015, 0.35955286, 0.38198876]),
 'score_time': array([0.00093985, 0.00095487, 0.00087571, 0.00090718, 0.00086689,
        0.00090098, 0.00090885, 0.00093699, 0.00090337, 0.0011282 ]),
 'test_score': array([0.89655172, 0.91954023, 0.91954023, 0.94827586, 0.95375723,
        0.93023256, 0.90697674, 0.94767442, 0.94186047, 0.9122807 ]),
 'train_score': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

In [641]:
cv_results['test_score'].mean()

0.9276690159211434

In [643]:
dt.fit(X, y)
dt.graph

{(): [5],
 (5,): [[0], 3, 3],
 ((5, 1), 3): [[0], 0, 0],
 ((5, 1), (3, 1), 0): [1, 4, 1, 1],
 ((5, 1), (3, 1), (0, 0), 1): [[0], [0], 4, 4],
 ((5, 1), (3, 1), (0, 0), (1, 2), 4): [[0], 2, [1]],
 ((5, 1), (3, 1), (0, 0), (1, 2), (4, 1), 2): [[0], [0], [1], [1]],
 ((5, 1), (3, 1), (0, 0), (1, 3), 4): [[0], 2, [1]],
 ((5, 1), (3, 1), (0, 0), (1, 3), (4, 1), 2): [[0], [0], [1], [1]],
 ((5, 1), (3, 1), (0, 1), 4): [[0], 2, 1],
 ((5, 1), (3, 1), (0, 1), (4, 1), 2): [[0], [0], 1, 1],
 ((5, 1), (3, 1), (0, 1), (4, 1), (2, 2), 1): [[0], [1], [1], [1]],
 ((5, 1), (3, 1), (0, 1), (4, 1), (2, 3), 1): [[0], [1], [1], [1]],
 ((5, 1), (3, 1), (0, 1), (4, 2), 1): [[0], [1], [1], [1]],
 ((5, 1), (3, 1), (0, 2), 1): [4, 4, [1], 4],
 ((5, 1), (3, 1), (0, 2), (1, 0), 4): [[0], 2, [1]],
 ((5, 1), (3, 1), (0, 2), (1, 0), (4, 1), 2): [[0], [0], [1], [1]],
 ((5, 1), (3, 1), (0, 2), (1, 1), 4): [[0], 2, [1]],
 ((5, 1), (3, 1), (0, 2), (1, 1), (4, 1), 2): [[0], [0], [1], [1]],
 ((5, 1), (3, 1), (0, 2), (1, 3), 

# Voting

In [644]:
vote = arff.Arff(arff=r'voting.arff', label_count=1)
np.random.shuffle(vote[:])
X = vote[:, :-1]
y = vote[:, -1].reshape(-1, 1)
for i in range(X.shape[1]):
    X[:, i][np.isnan(X[:, i])] = 2
dt = DTClassifier()
cv_results = cross_validate(dt, X, y, scoring='accuracy', return_train_score=True, cv=10)
cv_results

{'fit_time': array([0.23411679, 0.20066595, 0.21653795, 0.21369505, 0.21226382,
        0.18733215, 0.18990588, 0.20674515, 0.21313   , 0.22836685]),
 'score_time': array([0.00071216, 0.00039983, 0.0003891 , 0.00039673, 0.00043392,
        0.00035882, 0.00036812, 0.00052404, 0.00038719, 0.00047088]),
 'test_score': array([0.88636364, 0.86363636, 1.        , 0.95454545, 1.        ,
        0.93181818, 0.93181818, 0.93023256, 0.92857143, 0.97619048]),
 'train_score': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

In [645]:
cv_results['test_score'].mean()

0.9403176281083259

In [646]:
dt.fit(X, y)
dt.graph

{(): [3],
 (3,): [2, 10, 8],
 ((3, 0), 2): [11, [0], [0]],
 ((3, 0), (2, 0), 11): [10, [0], [1]],
 ((3, 0), (2, 0), (11, 0), 10): [5, [0], [0]],
 ((3, 0), (2, 0), (11, 0), (10, 0), 5): [13, [0], [0]],
 ((3, 0), (2, 0), (11, 0), (10, 0), (5, 0), 13): [[0], [1], [0]],
 ((3, 1), 10): [14, 2, [1]],
 ((3, 1), (10, 0), 14): [2, 9, [1]],
 ((3, 1), (10, 0), (14, 0), 2): [[1], 15, [1]],
 ((3, 1), (10, 0), (14, 0), (2, 1), 15): [[1], [1], 0],
 ((3, 1), (10, 0), (14, 0), (2, 1), (15, 2), 0): [[0], [1], [0]],
 ((3, 1), (10, 0), (14, 1), 9): [15, [1], [1]],
 ((3, 1), (10, 0), (14, 1), (9, 0), 15): [[1], [0], 1],
 ((3, 1), (10, 0), (14, 1), (9, 0), (15, 2), 1): [[0], [1], [0]],
 ((3, 1), (10, 1), 2): [4, 6, [0]],
 ((3, 1), (10, 1), (2, 0), 4): [[0], 15, [1]],
 ((3, 1), (10, 1), (2, 0), (4, 1), 15): [12, [1], 0],
 ((3, 1), (10, 1), (2, 0), (4, 1), (15, 0), 12): [[0], 1, [1]],
 ((3, 1), (10, 1), (2, 0), (4, 1), (15, 0), (12, 1), 1): [0, [1], [1]],
 ((3, 1), (10, 1), (2, 0), (4, 1), (15, 0), (12, 1), (

# Scikit-learn

In [648]:
from sklearn.tree import DecisionTreeClassifier as DTC

## Cars

In [701]:
cars = arff.Arff(arff=r'cars.arff', label_count=4)
np.random.shuffle(cars[:])
X = cars[:, :-1]
y = cars[:, -1].reshape(-1, 1)
dt = DTC(criterion="entropy", random_state=0)
cv_results = cross_validate(dt, X, y, scoring='accuracy', return_train_score=True, cv=10)
print(cv_results['test_score'])
print(cv_results['test_score'].mean())

dt = DTC(random_state=0)
cv_results = cross_validate(dt, X, y, scoring='accuracy', return_train_score=True, cv=10)
print(cv_results['test_score'])
print(cv_results['test_score'].mean())

dt = DTC(max_depth=10, min_samples_split=3, random_state=0)
cv_results = cross_validate(dt, X, y, scoring='accuracy', return_train_score=True, cv=10)
print(cv_results['test_score'])
print(cv_results['test_score'].mean())

[0.96551724 0.97126437 0.99425287 0.97701149 0.96531792 0.97674419
 0.97674419 0.97674419 0.98837209 0.97660819]
0.9768576734383931
[0.97701149 0.97701149 0.99425287 0.98850575 0.97687861 0.96511628
 0.97674419 0.97674419 0.98837209 0.98830409]
0.9808941059665465
[0.95977011 0.98275862 0.98275862 0.97126437 0.97687861 0.97674419
 0.98255814 0.97674419 0.97674419 0.97660819]
0.9762829221663616


## Voting

In [710]:
vote = arff.Arff(arff=r'cars.arff', label_count=4)
np.random.shuffle(vote[:])
X = vote[:, :-1]
y = vote[:, -1].reshape(-1, 1)
dt = DTC(criterion="entropy", random_state=0)
cv_results = cross_validate(dt, X, y, scoring='accuracy', return_train_score=True, cv=10)
print(cv_results['test_score'])
print(cv_results['test_score'].mean())

dt = DTC(random_state=0)
cv_results = cross_validate(dt, X, y, scoring='accuracy', return_train_score=True, cv=10)
print(cv_results['test_score'])
print(cv_results['test_score'].mean())

dt = DTC(min_impurity_decrease=0.001, random_state=0)
cv_results = cross_validate(dt, X, y, scoring='accuracy', return_train_score=True, cv=10)
print(cv_results['test_score'])
print(cv_results['test_score'].mean())

[0.97126437 0.98275862 0.99425287 0.97126437 0.97687861 0.98255814
 0.98837209 0.99418605 0.98255814 0.98245614]
0.9826549401557347
[0.98850575 0.98275862 1.         0.97126437 0.98265896 0.97674419
 0.99418605 0.99418605 0.98837209 0.97660819]
0.9855284254397283
[0.98275862 0.95402299 0.96551724 0.95402299 0.96531792 0.96511628
 0.97674419 0.95930233 0.99418605 0.94736842]
0.9664357016417539


## Covtype

In [777]:
from sklearn.datasets import fetch_covtype
from sklearn import tree

In [732]:
data = fetch_covtype()
X = data.data
y = data.target.reshape(-1, 1)

In [739]:
Xy = np.hstack((X, y))
np.random.shuffle(Xy)
X = Xy[:, :-1]
y = Xy[:, -1].reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [779]:
dt = DTC(max_depth=50, criterion="entropy")
dt.fit(X_train, y_train)
tree.export_graphviz(dt, out_file="tree.dot")
dt.score(X_test, y_test)

0.9393416100605838

In [771]:
dt = DTC(max_depth=50, criterion="gini")
dt.fit(X_train, y_train)
dt.score(X_test, y_test)

0.9332488066825776

In [781]:
dt = DTC(max_depth=3, criterion="entropy")
dt.fit(X_train, y_train)
tree.export_graphviz(dt, out_file="tree.dot")